Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
# Make environment
env = rlcard.make('limit-holdem', config={'seed': 0})
eval_env = rlcard.make('limit-holdem', config={'seed': 0})

In [5]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 100_000
evaluate_num = 10_000
episode_num = 10_000_000

In [6]:
# The paths for saving the logs and learning curves
log_dir = './experiments/limit_holdem_dqn_result/'

In [7]:
# Set a global seed
set_global_seed(0)

In [8]:
agent = DQNAgent(
    env.action_num,
    env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=64,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[1024, 512, 1024, 512]
)

random_agent = RandomAgent(action_num=eval_env.action_num)
env.set_agents([agent, agent])
eval_env.set_agents([agent, random_agent])

In [9]:
# Init a Logger to plot the learning curve
logger = Logger(log_dir)

In [10]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(agent.weight_updates,
                               tournament(eval_env, evaluate_num)[0])


----------------------------------------
  timestep     |  0
  reward       |  0.93775
----------------------------------------


KeyboardInterrupt: 

In [ ]:
# Close files in the logger
logger.close_files()

In [ ]:
# Plot the learning curve
logger.plot('DQN')

In [ ]:
# Save model
save_dir = 'models/limit_holdem_dqn'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
state_dict = agent.get_state_dict()
torch.save(state_dict, os.path.join(save_dir, 'model.pth'))

In [ ]:
eval_env_dqn_dqn = rlcard.make('limit-holdem', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)